# Practice notebook for regression analysis with dependent data in NHANES

This notebook will give you the opportunity to perform some analyses
using the regression methods for dependent data that we are focusing
on in this week of the course.

Enter the code in response to each question in the boxes labeled "enter your code here".
Then enter your responses to the questions in the boxes labeled "Type
Markdown and Latex".

This notebook is based on the NHANES case study notebook for
regression with dependent data.  Most of the code that you will need
to write below is very similar to code that appears in the case study
notebook.  You will need to edit code from that notebook in small ways
to adapt it to the prompts below.

To get started, we will use the same module imports and read the data
in the same way as we did in the case study:

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import numpy as np

url = "nhanes_2015_2016.csv"
da = pd.read_csv(url)

# Drop unused columns, drop rows with any missing values.
vars = ["BPXSY1", "RIDAGEYR", "RIAGENDR", "RIDRETH1", "DMDEDUC2", "BMXBMI",
        "SMQ020", "SDMVSTRA", "SDMVPSU", "BPXDI1"]
da = da[vars].dropna()

# This is the grouping variable
da["group"] = 10*da.SDMVSTRA + da.SDMVPSU

## Question 1: 

Build a marginal linear model using GEE for the first measurement of diastolic blood pressure (`BPXDI1`), accounting for the grouping variable defined above.  This initial model should have no covariates, and will be used to assess the ICC of this blood pressure measure.

In [2]:
model = sm.GEE.from_formula("BPXDI1 ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
print(result.cov_struct.summary())

model = sm.GEE.from_formula("BPXSY1 ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
print(result.cov_struct.summary())


The correlation between two observations in the same cluster is 0.031
The correlation between two observations in the same cluster is 0.030


__Q1a.__ What is the ICC for diastolic blood pressure?  What can you
  conclude by comparing it to the ICC for systolic blood pressure?

Diastolic blood pressure ICC = 0.031, which is negelible marginally strong than Systolic blood pressure
Systolic blood presseue ICC = 0.030

## Question 2: 

Take your model from question 1, and add gender, age, and BMI to it as covariates.

In [3]:
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})
model = sm.GEE.from_formula("BPXDI1 ~ RIAGENDRx + BMXBMI + RIDAGEYR", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.030


__Q2a.__ What is the ICC for this model?  What can you conclude by comparing it to the ICC for the model that you fit in question 1?

ICC remains at 0.030

## Question 3: 

Split the data into separate datasets for females and for males and fit two separate marginal linear models for diastolic blood pressure, one only for females, and one only for males.

In [4]:
da_f = da[da.RIAGENDRx == "Female"]
da_m = da[da.RIAGENDRx == "Male"]
model = sm.GEE.from_formula("BPXDI1 ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da_f)
result = model.fit()
print(result.cov_struct.summary())
model = sm.GEE.from_formula("BPXDI1 ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da_m)
result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.029
The correlation between two observations in the same cluster is 0.035


__Q3a.__ What do you learn by comparing these two fitted models?

ICC for female is 0.029 which is lower than male of 0.035 
The ICC takes on values from 0 to 1, 
1 corresponding to "perfect clustering" -- the values within a cluster are identical, 
0 corresponding to "perfect independence" -- the mean value within each cluster is identical across all the clusters.

## Question 4: 

Using the entire data set, fit a multilevel model for diastolic blood pressure, predicted by age, gender, BMI, and educational attainment.  Include a random intercept for groups.

In [8]:
model = sm.MixedLM.from_formula("BPXDI1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(DMDEDUC2)",
           groups="group", data=da)
da["group1"] = 10*da.SDMVSTRA + da.SDMVPSU + 10
result = model.fit()
print(result.summary())

            Mixed Linear Model Regression Results
Model:               MixedLM  Dependent Variable:  BPXDI1     
No. Observations:    5102     Method:              REML       
No. Groups:          30       Scale:               154.2055   
Min. group size:     106      Log-Likelihood:      -20115.7588
Max. group size:     226      Converged:           Yes        
Mean group size:     170.1                                    
--------------------------------------------------------------
                   Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
--------------------------------------------------------------
Intercept          64.688    1.153 56.123 0.000  62.429 66.947
RIAGENDRx[T.Male]   2.756    0.351  7.860 0.000   2.069  3.444
C(DMDEDUC2)[T.2.0]  0.736    0.726  1.013 0.311  -0.688  2.159
C(DMDEDUC2)[T.3.0] -0.139    0.644 -0.216 0.829  -1.401  1.122
C(DMDEDUC2)[T.4.0]  0.527    0.618  0.852 0.394  -0.685  1.739
C(DMDEDUC2)[T.5.0]  0.955    0.641  1.489 0.137  -0.302  2.212
C(DMD

__Q4a.__ How would you describe the strength of the clustering in this analysis?

__Q4b:__ Include a random intercept for age, and describe your findings.

In [6]:
da["age_cen"] = da.groupby("group").RIDAGEYR.transform(lambda x:x-x.mean())
model = sm.MixedLM.from_formula("BPXDI1 ~ age_cen + RIAGENDRx + BMXBMI + C(DMDEDUC2)",
           groups="group", re_formula= "1+age_cen", data=da)
result = model.fit()
print(result.summary())

             Mixed Linear Model Regression Results
Model:              MixedLM   Dependent Variable:   BPXDI1     
No. Observations:   5102      Method:               REML       
No. Groups:         30        Scale:                153.1451   
Min. group size:    106       Log-Likelihood:       -20110.2808
Max. group size:    226       Converged:            Yes        
Mean group size:    170.1                                      
---------------------------------------------------------------
                    Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
---------------------------------------------------------------
Intercept           62.806    1.014 61.952 0.000  60.819 64.793
RIAGENDRx[T.Male]    2.756    0.350  7.874 0.000   2.070  3.442
C(DMDEDUC2)[T.2.0]   0.709    0.727  0.976 0.329  -0.716  2.134
C(DMDEDUC2)[T.3.0]  -0.147    0.646 -0.227 0.820  -1.412  1.119
C(DMDEDUC2)[T.4.0]   0.491    0.621  0.791 0.429  -0.726  1.708
C(DMDEDUC2)[T.5.0]   0.921    0.644  1.430 0.153  -0.

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
